In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from plotting_utils import *
import altair as alt
import altair_saver
alt.data_transformers.disable_max_rows()

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('hallway_v2*', subdir='exploration'),
    load_jobs('hallway_v2*no_explorationTrue', subdir='exploration', title="SAC"),
    load_jobs('hallway_v2*no_explorationFalse', subdir='exploration', title="SAC + UFO"),
#     load_jobs('hallway_v3*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
for task in ['velocity_1', 'velocity_4']:
    chart = make_base_chart(
        subset[subset.task == task], 
        title="", 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/hallway_{task}.pdf')
chart

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('hallway_distractor_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
# #     load_jobs('hallway_distractor_v10*no_explorationFalse*discount0.97*', subdir='exploration', title="4 u pdates, novelty discount 0.97"),
    load_jobs('hallway_distractor_v10*no_explorationFalse*discount0.99*', subdir='exploration', title="SAC + UFO"),
]
data = pd.concat(jobs, sort=False)

N = 100
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset,
    title="", 
    color='title',
#     color='title',
#     color='policy_updates_per_step:O',
#     opacity='use_exploration',z
    window=3,
)

chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

altair_saver.save(chart, 'results/hallway_4_distractor.pdf')
chart

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('hallway_inverse_distractor_v2*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('hallway_inverse_distractor_v2*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="", 
    color='title',
    window=1,
)

chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

altair_saver.save(chart, 'results/hallway_inverse_distractor.pdf')
chart

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('bice_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
#     load_jobs('bice_v1*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),

    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('bice_v6*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)
data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
charts = []
for env in ['ball_in_cup', 'ball_in_cup_explore']:
    chart = make_base_chart(
        subset[subset.env == env], 
        title="", 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/{env}.pdf')
    charts.append(chart)
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('reacher_explore_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
#     load_jobs('reacher_explore_v3*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
    
#     load_jobs('reacher_explore_v4*no_explorationTrue*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title="SAC"),
#     load_jobs('reacher_explore_v4*no_explorationFalse*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title="SAC + UFO"),
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('reacher_v10*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)
data = data[(data.env == 'reacher') | (data.env == 'reacher_explore')]

N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]

charts = []
for task in ['hard', 'hard_narrow_init']:
    chart = make_base_chart(
        subset[subset.task == task], 
        title="", 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/reacher_{task}.pdf')
    charts.append(chart)
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('fex_hard_v3*no_explorationTrue*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title="SAC"),
#     load_jobs('fex_hard_v3*no_explorationFalse*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title="SAC + UFO"),
#     load_jobs('finger_v2*no_explorationTrue*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title="SAC"),
#     load_jobs('finger_v2*no_explorationFalse*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title="SAC + UFO"),
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('finger_all_v6*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
#     load_jobs('finger_all_v5*no_explorationTrue*', subdir='exploration', title="SAC",
#               root='/home/will/vlg5/exploration-reimplement'),
#     load_jobs('finger_all_v5*no_explorationFalse*', subdir='exploration', title="SAC + UFO",
#               root='/home/will/vlg5/exploration-reimplement'),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
subset = subset[(subset['episode'] <= N)]
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]

charts = []
for env in ['finger', 'finger_explore']:
    chart = make_base_chart(
        subset[subset.env == env], 
        title="", 
        color='title',
        window=30,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/{env}_turn_hard.pdf')
    charts.append(chart)
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
#     load_jobs('wex_walk_narrow_sparse_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
#     load_jobs('wex_walk_narrow_sparse_v1*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
    
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('walker_walk_v1*no_explorationFalse*', subdir='exploration', title="SAC + UFO"),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)

N = 400
subset = data
subset = subset[(subset['episode'] <= N)]
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]

charts = []
for env in ['walker', 'walker_explore']:
    chart = make_base_chart(
        subset[subset.env == env], 
        title="", 
        color='title',
        window=10,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/{env}_walk.pdf')
    charts.append(chart)
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)
# data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]

# N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
# subset = subset[(subset['episode'] <= N)]

Ns = {
    'ball_in_cup': 500, 'ball_in_cup_explore': 500,
    'reacher': 500, 'reacher_explore': 500,
    'finger': 1000, 'finger_explore': 1000,
    'walker': 500, 'walker_explore': 500,
}

charts = []
for env in sorted(subset.env.unique()):
    N = Ns[env]
    chart = make_base_chart(
        subset[(subset.env == env) & (subset.episode <= N)], 
#         title=env, 
        title='', 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/neurips_{env}.pdf')
    charts.append(chart)
alt.vconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)

# Ns = {
#     'ball_in_cup': 500, 'ball_in_cup_explore': 500,
#     'reacher': 500, 'reacher_explore': 500,
#     'finger': 1000, 'finger_explore': 1000,
#     'walker': 500, 'walker_explore': 500,
# }

env_names = {
    'ball_in_cup': 'BIC', 'ball_in_cup_explore': 'BIC explore',
    'reacher': 'Reacher', 'reacher_explore': 'Reacher explore',
    'finger': 'Finger', 'finger_explore': 'Finger explore',
    'walker': 'Walker', 'walker_explore': 'Walker explore',
}

subsets = []
for env in sorted(data.env.unique()):
#     N = Ns[env] / 5
    N = 100    
    env_subset = data[(data.env == env) & (data.test == True) &
                      (data.episode <= N + 10) & (N - 10 < data.episode)].copy()
    env_subset['pretty_env'] = env_names[env]
    subsets.append(env_subset)
subset = pd.concat(subsets, sort=False)

chart = alt.Chart(subset, width=100, height=300).mark_bar(size=22).encode(
    x=alt.X('title', title='Algorithm', axis=None),
    y=alt.Y('mean(score)', title='Evaluation score', scale=alt.Scale(domain=(0,1000)), axis=alt.Axis(domainOpacity=0)),
    color=alt.Color('title', title='Algorithm', legend=alt.Legend(orient='none', legendX=950, legendY=0)),
    column=alt.Column('pretty_env', title='Environment', 
                      spacing=20, header=alt.Header(labelOrient='bottom', titleOrient='bottom', titleFontSize=24, labelFontSize=20)),
).configure_view(stroke='transparent')


# chart = alt.Chart(subset).mark_bar(opacity=1).encode(
#     column = alt.Column('env:O', spacing = 5, header = alt.Header(labelOrient = "bottom")),
#     x =alt.X('title', sort = ["SAC", "SAC + BBE", "SAC + DEX"],  axis=None),
#     y =alt.Y('mean(score):Q'),
#     color= alt.Color('title')
# ).configure_view(stroke='transparent')

# altair_saver.save(chart, 'results/control_suite_summary.pdf', method='node')
chart.display()

In [ ]:
altair_saver.save?

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('grid40_warmstart_v6*no_explorationTrue*', root='/home/will/vlg5/exploration-reimplement', subdir='exploration', title='DDQN'),
    load_jobs('grid40_warmstart_v6*no_explorationFalse*', root='/home/will/vlg5/exploration-reimplement', subdir='exploration', title='DDQN + DEEP'),
#     load_jobs('grid40_warmstart_v6*bonus_scale0.1*', root='/home/will/vlg5/exploration-reimplement', subdir='bbe', title='DDQN + BBE, 0.1'),
    load_jobs('grid40_warmstart_v6*bonus_scale1', root='/home/will/vlg5/exploration-reimplement', subdir='bbe', title='DDQN + BBE'),    

]
data = pd.concat(jobs, sort=False)

N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset.episode = subset.episode - 20
subset = subset[(subset.episode > 0) & (subset.episode <= N)]

chart = make_base_chart(
    subset,
    title=f"",
    color='title',
    window=10,
)
chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,23))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

altair_saver.save(chart, f'results/grid40_warmstart.pdf')
chart

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('hallway_distractors_v13*no_explorationTrue*', subdir='exploration', title='SAC'),
    load_jobs('hallway_distractors_v13*no_explorationFalse*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]

charts = []
for task in ['velocity_4_distractor']:
    chart = make_base_chart(
        subset[subset.task == task], 
        title=f"",
        color='title',
#         strokeDash='task',
#         detail='task',
        window=5,
    )
    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right'),
                               scale=alt.Scale(domain=['SAC', 'SAC + DEEP'], range=['#66c2a5', '#8da0cb'])))
    charts.append(chart)
    altair_saver.save(chart, f'results/hallway_{task}.pdf')
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('hallway_distractors_v13*no_explorationTrue*', subdir='exploration', title='SAC'),
    load_jobs('hallway_distractors_v13*no_explorationFalse*', subdir='exploration', title='SAC + DEEP'),
]
data = pd.concat(jobs, sort=False)

N = 100
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]

charts = []
for task in ['velocity_4_inverse_distractor']:
    chart = make_base_chart(
        subset[subset.task == task], 
        title=f"",
        color='title',
#         strokeDash='task',
#         detail='task',
        window=5,
    )
    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
               y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
               color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right'),
                               scale=alt.Scale(domain=['SAC', 'SAC + DEEP'], range=['#66c2a5', '#8da0cb'])))
    charts.append(chart)
    altair_saver.save(chart, f'results/hallway_{task}.pdf')
alt.hconcat(*charts)

In [ ]:
alt.themes.enable('publication')
jobs = [
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('finger_all_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('walker_walk_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
    load_jobs('wex_walk_narrow_sparse_v3*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bice_v6*no_explorationTrue*', subdir='exploration', title="SAC"),
    
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
    load_jobs('bbe_all_v3*', subdir='bbe', title='SAC + BBE'),
    load_jobs('ufo_all_v2*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('ufo_all_v3*', subdir='exploration', title='SAC + DEEP'),
    load_jobs('sacqex_all_v2*', subdir='sac_qex', title='SAC 2Q'),
]
data = pd.concat(jobs, sort=False)
# data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]

# N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
# subset = subset[(subset['episode'] <= N)]

Ns = {
    'ball_in_cup': 500, 'ball_in_cup_explore': 500,
    'reacher': 500, 'reacher_explore': 500,
    'finger': 1000, 'finger_explore': 1000,
    'walker': 500, 'walker_explore': 500,
}

charts = []
for env in sorted(subset.env.unique()):
    N = Ns[env]
    chart = make_base_chart(
        subset[(subset.env == env) & (subset.episode <= N)], 
#         title=env, 
        title='', 
        color='title',
        window=20,
    )

    chart = plot_with_bars(chart, 'smoothed_score', test=True, title_flag='',
                   y_args={'title': 'Evaluation score', 'scale': alt.Scale(domain=(0,1000))}, 
                   color=alt.Color('title', legend=alt.Legend(title='Algorithm', orient='bottom-right')))

    altair_saver.save(chart, f'results/neurips_SAC2Q_{env}.pdf')
    charts.append(chart)
alt.vconcat(*charts)